Podklady pro články Anety Martínkové na iROZHLAS.cz v prosinci 2023.

In [1]:
import os
import pandas as pd

In [2]:
from src.tabulka import tabulka
from src.graf import graf

In [3]:
cz = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))

In [4]:
cz_interpreti = cz['interpret_hlavni'].drop_duplicates().to_list()

## vs. Billboard 100

In [5]:
b100 = pd.read_csv(os.path.join("data_b100","b100.csv"))

In [6]:
b100 = b100[b100['rok'] >= 2006]

In [7]:
us_interpreti = b100['interpret_hlavni'].drop_duplicates().to_list()

In [8]:
interpreti = cz_interpreti + us_interpreti

In [9]:
def nejmensispolecny(x):
    x = x.lower().split('(')[0]
    if x[0:3] == "the":
        x = x.replace("the ","")
    return x

In [10]:
interpreti_slovnik = {}
for i in interpreti:
    malym = nejmensispolecny(i)
    interpreti_slovnik[malym] = i

In [11]:
cz = cz.sort_values(by = ['rok','tyden','pozice'], ascending=True)

In [12]:
cz['interpret_hlavni'] = cz['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))
cz['track'] = cz['track'].apply(lambda x: nejmensispolecny(x))

In [13]:
cz['wk'] = cz['week'].rank(method='dense').astype(int)

In [14]:
b100['week'] = b100.apply(lambda row: f"""{row['rok']}-{row['tyden']:02}""", axis = 1)

In [15]:
b100 = b100.rename(columns = {'song': 'track'})

In [16]:
b100['interpret_hlavni'] = b100['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))
b100['track'] = b100['track'].apply(lambda x: nejmensispolecny(x))

In [17]:
cz = cz.sort_values(by="week")
b100 = b100.sort_values(by="week")

In [18]:
b100.tail()

,date,rank,track,artist,last-week,peak-rank,weeks-on-board,rok,datum,tyden,interpret_hlavni,week
317414,2021-11-06,28,let's go brandon,Bryson Gray Featuring Tyson James & Chandler C...,NaN,28,1,2021,11-06,45,bryson gray,2021-45
317413,2021-11-06,27,moth to a flame,Swedish House Mafia & The Weeknd,NaN,27,1,2021,11-06,45,swedish house mafia,2021-45
317411,2021-11-06,25,chasing after you,Ryan Hurd With Maren Morris,25.0,23,26,2021,11-06,45,ryan hurd,2021-45
317422,2021-11-06,36,buy dirt,Jordan Davis Featuring Luke Bryan,34.0,34,12,2021,11-06,45,jordan davis,2021-45
317486,2021-11-06,100,nevada,YoungBoy Never Broke Again,NaN,58,4,2021,11-06,45,youngboy never broke again,2021-45


In [19]:
cz = cz[cz['week'] <= '2021-45']

In [20]:
[x for x in b100['week'].drop_duplicates().to_list() if x not in cz['week'].drop_duplicates().to_list()]

['2006-51',
 '2007-52',
 '2009-51',
 '2010-50',
 '2011-26',
 '2011-51',
 '2011-53',
 '2012-52',
 '2013-52',
 '2014-33',
 '2014-51',
 '2015-52',
 '2016-51',
 '2016-53',
 '2017-51',
 '2018-08',
 '2018-51',
 '2019-30',
 '2020-01']

Pozor, seznamy týdnů se úplně nepřekrývají, srovnání může lítat o týden.

### Porovnání interpretů

In [21]:
am_jednicky = pd.Series(b100[b100['rank'] == 1].groupby('interpret_hlavni')['track'].nunique())
am_jednicky = am_jednicky[am_jednicky >= 3]
am_jednicky = am_jednicky.index.to_list()

In [22]:
am_jednicky

['adele',
 'ariana grande',
 'beyoncé',
 'black eyed peas',
 'britney spears',
 'bruno mars',
 'bts',
 'cardi b',
 'drake',
 'eminem',
 'fergie',
 'flo rida',
 'justin bieber',
 'justin timberlake',
 'katy perry',
 'lady gaga',
 'mariah carey',
 'maroon 5',
 'p!nk',
 'post malone',
 'rihanna',
 'taylor swift',
 'travis scott',
 'weeknd']

In [23]:
americke_jednicky = b100[b100['rank'] == 1]['interpret_hlavni'].drop_duplicates().to_list()

In [24]:
americke_jednicky_v_cesku = pd.Series(cz[cz['interpret_hlavni'].isin(am_jednicky)].groupby('interpret_hlavni')['pozice'].min().reindex(am_jednicky).sort_values())

In [25]:
americke_jednicky_v_cesku = pd.DataFrame(americke_jednicky_v_cesku[americke_jednicky_v_cesku != 1]).reset_index().rename(columns = {'interpret_hlavni':'Interpreti','pozice':'#'}).replace({"Interpreti": interpreti_slovnik})

In [26]:
tabulka(americke_jednicky_v_cesku,titulek='V USA 2× nebo víckrát jedničkou, v Česku pod vrcholem', podtitulek='Data z Billboard 100 jsou od roku 2006 do podzimu 2021, česká data do listopadu 2023.')

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>V USA 2× nebo víckrát jedničkou, v Česku pod vrcholem</caption><thead><tr style="text-align: right;"><th class="text-nowrap">Interpreti</th><th class="text-nowrap">#</th></tr></thead><tbody class="text-sm"><tr><td>Maroon 5</td><td>2.0</td></tr><tr><td>Beyoncé</td><td>2.0</td></tr><tr><td>Eminem</td><td>2.0</td></tr><tr><td>Ariana Grande</td><td>5.0</td></tr><tr><td>Britney Spears</td><td>6.0</td></tr><tr><td>Post Malone</td><td>7.0</td></tr><tr><td>Mariah Carey</td><td>11.0</td></tr><tr><td>Drake</td><td>26.0</td></tr><tr><td>Cardi B</td><td>44.0</td></tr><tr><td>BTS</td><td>NaN</td></tr><tr><td>Travis Scott</td><td>NaN</td></tr></tbody><tfoot><tr style="text-align: center;"><td colspan=2>Data z Billboard 100 jsou od roku 2006 do podzimu 2021, česká data do listopadu 2023.</td></tr></tfoot></table>'

## vs. stream

In [27]:
df = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))
df['week'] = df.apply(lambda x: f"""{x['rok']}-{x['tyden']}""", axis = 1)
df = df[df['rok'] >= 2015]

In [28]:
df['track'] = df['track'].apply(lambda x: nejmensispolecny(x))
df['interpret_hlavni'] = df['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))

In [29]:
stream = pd.read_csv(os.path.join("data","CZ_SINGLES_DIGITAL_TOP_100_tydenni.csv"))
stream = stream[stream['rok'] >= 2015]
stream['week'] = stream.apply(lambda x: f"""{x['rok']}-{x['tyden']}""", axis = 1)

In [30]:
stream_interpreti = stream['interpret_hlavni'].drop_duplicates().to_list()

In [31]:
for s in stream_interpreti:
    malym = nejmensispolecny(s)
    interpreti_slovnik[malym] = s

In [32]:
interpreti = interpreti + stream_interpreti

In [33]:
stream['track'] = stream['track'].apply(lambda x: nejmensispolecny(x))
stream['interpret_hlavni'] = stream['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))

In [34]:
df['w'] = df['week'].rank(method='dense').astype(int)

In [35]:
stream['w'] = stream['week'].rank(method='dense').astype(int)

### Porovnání písní: která bodovala dřív online (-) a která v rádiu (+)

In [36]:
jednicky_radio = df[df['pozice'] == 1].groupby(['interpret_hlavni','track'])['w'].min()

In [37]:
jednicky_stream = stream[stream['pozice'] == 1].groupby(['interpret_hlavni','track'])['w'].min()

In [38]:
jednicky_stream

interpret_hlavni  track         
24kgoldn          mood              286
6ix9ine           gooba             267
adele             easy on me        343
                  hello              39
alan walker       faded              60
                                   ... 
yzomandias        designer flow     296
                  free karlo        398
                  rodinnej typ      354
                  rolls             282
zayn              dusk till dawn    140
Name: w, Length: 96, dtype: int32

In [39]:
(jednicky_stream - jednicky_radio).dropna().sort_values().head()

interpret_hlavni   track            
imagine dragons    bad liar            -53.0
hozier             take me to church   -33.0
twenty one pilots  heathens            -15.0
ed sheeran         i don't care        -13.0
shawn mendes       senorita            -12.0
Name: w, dtype: float64

In [40]:
(jednicky_stream - jednicky_radio).dropna().sort_values().tail()

interpret_hlavni  track              
justin bieber     love yourself          -1.0
ed sheeran        shape of you            0.0
alan walker       faded                   0.0
ellie goulding    love me like you do     0.0
ava max           sweet but psycho       11.0
Name: w, dtype: float64

In [41]:
(jednicky_stream - jednicky_radio).dropna().mean()

-9.666666666666666

In [42]:
dodeseti_radio = df[df['pozice'] <= 10].groupby(['interpret_hlavni','track'])['w'].min()
dodeseti_stream = stream[stream['pozice'] <= 10].groupby(['interpret_hlavni','track'])['w'].min()
(dodeseti_stream - dodeseti_radio).dropna().sort_values()

interpret_hlavni  track               
ed sheeran        happier                -87.0
sam smith         too good at goodbyes   -67.0
shawn mendes      stitches               -61.0
justin bieber     sorry                  -58.0
billie eilish     therefore i am         -58.0
                                          ... 
julia michaels    issues                  28.0
alan walker       alone                   31.0
ofenbach          katchi                  38.0
sheppard          geronimo                44.0
david guetta      dangerous               44.0
Name: w, Length: 167, dtype: float64

In [43]:
(dodeseti_stream - dodeseti_radio).dropna().mean()

-9.479041916167665

In [44]:
(dodeseti_stream - dodeseti_radio).dropna().median()

-9.0

In [45]:
dosta_radio = df[df['pozice'] <= 100].groupby(['interpret_hlavni','track'])['w'].min()
dosta_stream = stream[stream['pozice'] <= 100].groupby(['interpret_hlavni','track'])['w'].min()
(dosta_stream - dosta_radio).dropna().sort_values()

interpret_hlavni  track                    
taylor swift      cruel summer                -204.0
jubel             dancing in the moonlight    -103.0
martin garrix     high on life                 -98.0
ed sheeran        happier                      -69.0
p!nk              just give me a reason        -65.0
                                               ...  
coldplay          viva la vida                 263.0
olga lounová      jsem optimista               313.0
jelen             magdaléna                    363.0
kabát             western boogie               378.0
amy macdonald     this is the life             411.0
Name: w, Length: 874, dtype: float64

In [46]:
(dosta_stream - dosta_radio).dropna().mean()

-1.1418764302059496

In [47]:
(dosta_stream - dosta_radio).dropna().median()

-4.0

Polovina písniček je o 4 týdny nebo více dřív populární online než v rádiu.

### co lidé stahují a streamují vs. co hrají djové v rádiu

In [48]:
radio = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))

In [49]:
stream['rok'].min()

2015

In [50]:
radio = radio[radio['rok'] >= stream['rok'].min()]

In [51]:
radio['interpret_hlavni'] = radio['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))

#### Porovnání výskytů interpretů v top 100 rádio vs. stream od roku 2014

In [52]:
radio.groupby('interpret_hlavni').size().nlargest(10)

interpret_hlavni
mirai              1180
kryštof             872
marek ztracený      817
chinaski            786
slza                746
ed sheeran          666
xindl x             650
imagine dragons     592
robin schulz        492
alvaro soler        463
dtype: int64

In [53]:
stream.groupby('interpret_hlavni').size().nlargest(10)

interpret_hlavni
viktor sheen       1753
imagine dragons    1377
ed sheeran         1212
calin              1072
yzomandias          978
weeknd              700
nik tendo           695
kygo                623
ektor               531
chainsmokers        527
dtype: int64

In [54]:
pomer = (radio.groupby('interpret_hlavni').size() / stream.groupby('interpret_hlavni').size()).sort_values()
hvezdy_streamu = pd.DataFrame(pomer).reset_index().rename(columns={'interpret_hlavni':'Interpreti',0:'Poměr stream/rádio'}).head(10).replace({"Interpreti": interpreti_slovnik})
hvezdy_streamu['Poměr stream/rádio'] = 1 / hvezdy_streamu['Poměr stream/rádio']
hvezdy_streamu['Poměr stream/rádio'] = hvezdy_streamu['Poměr stream/rádio'].apply(lambda x: f"""{int(x)}×""")
hvezdy_radia = pd.DataFrame(pomer.nlargest(10)).reset_index().rename(columns={'interpret_hlavni':'Interpreti',0:'Poměr rádio/stream'}).replace({"Interpreti": interpreti_slovnik})
hvezdy_radia['Poměr rádio/stream'] = hvezdy_radia['Poměr rádio/stream'].apply(lambda x: f"""{int(x)}×""")

In [55]:
hvezdy_streamu

,Interpreti,Poměr stream/rádio
0,Calin,63×
1,James Hype,60×
2,Ben Cristovao,40×
3,Echosmith,37×
4,Matoma,37×
5,Passenger,31×
6,Rema,30×
7,Rae Sremmurd,25×
8,Kapitán Demo,23×
9,Marc E. Bassy,21×


In [56]:
tabulka(hvezdy_streamu, titulek="Koho lidé výrazně častěji poslouchají online než slyší v rádiu", podtitulek="Srovnání žebříčků top 100 radio a top 100 digital od roku 2015. Číslo vyjadřuje poměr mezi týdny v online a rádiových hitparádách.")

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>Koho lidé výrazně častěji poslouchají online než slyší v rádiu</caption><thead><tr style="text-align: right;"><th class="text-nowrap">Interpreti</th><th class="text-nowrap">Poměr stream/rádio</th></tr></thead><tbody class="text-sm"><tr><td>Calin</td><td>63×</td></tr><tr><td>James Hype</td><td>60×</td></tr><tr><td>Ben Cristovao</td><td>40×</td></tr><tr><td>Echosmith</td><td>37×</td></tr><tr><td>Matoma</td><td>37×</td></tr><tr><td>Passenger</td><td>31×</td></tr><tr><td>Rema</td><td>30×</td></tr><tr><td>Rae Sremmurd</td><td>25×</td></tr><tr><td>Kapitán Demo</td><td>23×</td></tr><tr><td>Marc E. Bassy</td><td>21×</td></tr></tbody><tfoot><tr style="text-align: center;"><td colspan=2>Srovnání žebříčků top 100 radio a top 100 digital od roku 2015. Číslo vyjadřuje poměr mezi týdny v online a rádiových hitparádách.</td></tr></tfoot></table>'

In [57]:
hvezdy_radia

,Interpreti,Poměr rádio/stream
0,Poetika,160×
1,Tomáš Klus,80×
2,Purple Disco Machine,74×
3,Imany,65×
4,David Stypka,63×
5,Bastille,59×
6,Voxel,57×
7,Robbie Williams,57×
8,Lake Malawi,54×
9,Avril Lavigne,49×


In [58]:
tabulka(hvezdy_radia, titulek="Co lidé výrazně častěji slyší v rádiu než poslouchají online", podtitulek="Srovnání žebříčků top 100 radio a top 100 digital od roku 2015. Číslo vyjadřuje poměr mezi týdny v rádiových a online hitparádách.")

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>Co lidé výrazně častěji slyší v rádiu než poslouchají online</caption><thead><tr style="text-align: right;"><th class="text-nowrap">Interpreti</th><th class="text-nowrap">Poměr rádio/stream</th></tr></thead><tbody class="text-sm"><tr><td>Poetika</td><td>160×</td></tr><tr><td>Tomáš Klus</td><td>80×</td></tr><tr><td>Purple Disco Machine</td><td>74×</td></tr><tr><td>Imany</td><td>65×</td></tr><tr><td>David Stypka</td><td>63×</td></tr><tr><td>Bastille</td><td>59×</td></tr><tr><td>Voxel</td><td>57×</td></tr><tr><td>Robbie Williams</td><td>57×</td></tr><tr><td>Lake Malawi</td><td>54×</td></tr><tr><td>Avril Lavigne</td><td>49×</td></tr></tbody><tfoot><tr style="text-align: center;"><td colspan=2>Srovnání žebříčků top 100 radio a top 100 digital od roku 2015. Číslo vyjadřuje poměr mezi týdny v rádiových a online hitparádách.</td></tr></tfoot></table>'

In [59]:
jenomonline = pd.DataFrame(stream[~stream['interpret_hlavni'].isin(radio['interpret_hlavni'].drop_duplicates().to_list())].groupby('interpret_hlavni')['week'].nunique().nlargest(10)).reset_index().rename(columns={'interpret_hlavni':'Interpreti','week':'Týdnů v digi hitparádě'}).replace({"Interpreti": interpreti_slovnik})

In [60]:
jenomonline

,Interpreti,Týdnů v digi hitparádě
0,Wohnout,248
1,Viktor Sheen,246
2,Yzomandias,245
3,Nik Tendo,225
4,Ektor,216
5,Škwor,157
6,Travis Scott,151
7,Earth,149
8,Ca$Hanova Bulhar,148
9,Neighbourhood,144


In [61]:
tabulka(hvezdy_radia, titulek="Online bodují, v rádiu zůstali pod stovkou nejhranějších", podtitulek="Data od roku 2015 do roku 2023.")

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>Online bodují, v rádiu zůstali pod stovkou nejhranějších</caption><thead><tr style="text-align: right;"><th class="text-nowrap">Interpreti</th><th class="text-nowrap">Poměr rádio/stream</th></tr></thead><tbody class="text-sm"><tr><td>Poetika</td><td>160×</td></tr><tr><td>Tomáš Klus</td><td>80×</td></tr><tr><td>Purple Disco Machine</td><td>74×</td></tr><tr><td>Imany</td><td>65×</td></tr><tr><td>David Stypka</td><td>63×</td></tr><tr><td>Bastille</td><td>59×</td></tr><tr><td>Voxel</td><td>57×</td></tr><tr><td>Robbie Williams</td><td>57×</td></tr><tr><td>Lake Malawi</td><td>54×</td></tr><tr><td>Avril Lavigne</td><td>49×</td></tr></tbody><tfoot><tr style="text-align: center;"><td colspan=2>Data od roku 2015 do roku 2023.</td></tr></tfoot></table>'